In [1]:
import pandas as pd
import numpy as np
import traceback

from esa_snappy import ProductIO
from esa_snappy import GeoPos
from esa_snappy import PixelPos

from glob import glob
from tqdm import tqdm
import os
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

# Suppress specific warning messages
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="org.esa.snap.core.dataio.dimap.DimapProductReader")


INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.


In [2]:

def check_image(mgrs):
    processed_mosaic = os.listdir("/data/ksa/01_Image_Acquisition/02_Processed_mosaic/")
    if(mgrs in processed_mosaic):
        return True
    else: 
        return False

def get_gc(path_to_sentinel_data):
    # path_to_sentinel_data = "/data/ksa/01_Image_Acquisition/02_Processed_mosaic/48MXU/20230501_20230512.dim"
    product_subset = ProductIO.readProduct(path_to_sentinel_data)
    gc = product_subset.getSceneGeoCoding()
    bands_names = list(product_subset.getBandNames())
    return product_subset,gc,bands_names

def get_values(gc, bands_names, lat, long):
    pixel_pos = gc.getPixelPos(GeoPos(lat, long), None)
    data = list()
    for i, band_name in enumerate(bands_names):
        temp_band = product_subset.getBand(band_name)
        width, height = temp_band.getRasterWidth(), temp_band.getRasterHeight()
        try:
            tmp = np.zeros(1)
            temp_band.readPixels(int(pixel_pos.x), int(pixel_pos.y), 1, 1, tmp)
            data.append(tmp[0])
            data_values = [float(val) for val in data]
        except Exception as e:
            print(band_name)
            print(width, height)
            print(int(pixel_pos.x), int(pixel_pos.y))
            print(e)
            traceback.print_exc()
    return data_values

# data_values
def get_value(gc, lat, long):    
    
    data = list()
    pixel_pos = gc.getPixelPos(GeoPos(lat, lon), None)
    data.append(lat).append(lon).append(int(pixel_pos.x)).append(int(pixel_pos.y))

    for i, band_name in enumerate(bands_names):
        temp_band = product_subset.getBand(band_name)
        width, height = temp_band.getRasterWidth(), temp_band.getRasterHeight()
        try:
            tmp = np.zeros(1)
            temp_band.readPixels(int(pixel_pos.x), int(pixel_pos.y), 1, 1, tmp)
            data.append(tmp[0])
        except Exception as e:
            print(band_name)
            print(width, height)
            print(int(pixel_pos.x), int(pixel_pos.y))
            print(e)
            traceback.print_exc()
    return data


In [3]:
# source : https://forum.step.esa.int/t/extracting-pixel-values-sentinel-2a-l2a-with-snap-or-snappy/29878/3


In [4]:
df_ksa = pd.read_csv("/data/raw/processed/cloned_points.csv")
# df_ksa.iloc[:1000].to_csv("cek.csv")

# df_ksa = pd.read_csv("/data/raw/processed/generated_points_unstable.csv")
df_ksa['idprov'] = df_ksa.idsegmen.astype('str').str[:2]
df_ksa['idkab'] = df_ksa.idsegmen.astype('str').str[:4]

df_ksa['index'] = [x.zfill(2) for x in df_ksa['index'].astype("str")]
df_ksa['idpoint'] = df_ksa.idsegmen.astype('str') + df_ksa.idsubsegmen.astype('str') + '#' + df_ksa['index']
df_ksa.head()

,iterx,itery,lat,long,index,idsegmen,strati,idsubsegmen,EASTING,NORTHING,100kmSQ_ID,GZD,MGRS,idprov,idkab,idpoint
0,1,1,2.381336,96.481115,01,110101001,S2,A1,200000mE,200000mN,KC,47N,47NKC,11,1101,110101001A1#01
1,1,2,2.381156,96.481115,02,110101001,S2,A1,200000mE,200000mN,KC,47N,47NKC,11,1101,110101001A1#02
2,1,3,2.380977,96.481115,03,110101001,S2,A1,200000mE,200000mN,KC,47N,47NKC,11,1101,110101001A1#03
3,1,4,2.380797,96.481115,04,110101001,S2,A1,200000mE,200000mN,KC,47N,47NKC,11,1101,110101001A1#04
4,1,5,2.380618,96.481115,05,110101001,S2,A1,200000mE,200000mN,KC,47N,47NKC,11,1101,110101001A1#05


In [5]:
idprov = "35"
df_ksa = df_ksa.query('idprov == "{0}"'.format(idprov))
df_ksa

,iterx,itery,lat,long,index,idsegmen,strati,idsubsegmen,EASTING,NORTHING,100kmSQ_ID,GZD,MGRS,idprov,idkab,idpoint
2743650,1,1,-8.092647,110.984621,01,350101001,S1,A1,400000mE,9100000mN,DM,49L,49LDM,35,3501,350101001A1#01
2743651,1,2,-8.092825,110.984621,02,350101001,S1,A1,400000mE,9100000mN,DM,49L,49LDM,35,3501,350101001A1#02
2743652,1,3,-8.093003,110.984621,03,350101001,S1,A1,400000mE,9100000mN,DM,49L,49LDM,35,3501,350101001A1#03
2743653,1,4,-8.093181,110.984621,04,350101001,S1,A1,400000mE,9100000mN,DM,49L,49LDM,35,3501,350101001A1#04
2743654,1,5,-8.093358,110.984621,05,350101001,S1,A1,400000mE,9100000mN,DM,49L,49LDM,35,3501,350101001A1#05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3829495,5,1,-7.861214,112.551459,21,357903038,S1,C3,600000mE,9100000mN,FM,49M,49MFM,35,3579,357903038C3#21
3829496,5,2,-7.861392,112.551459,22,357903038,S1,C3,600000mE,9100000mN,FM,49M,49MFM,35,3579,357903038C3#22
3829497,5,3,-7.861570,112.551459,23,357903038,S1,C3,600000mE,9100000mN,FM,49M,49MFM,35,3579,357903038C3#23
3829498,5,4,-7.861748,112.551459,24,357903038,S1,C3,600000mE,9100000mN,FM,49M,49MFM,35,3579,357903038C3#24


In [ ]:
# df_repair = pd.read_csv("/data/raw/processed/data_ksa_long_relabeled_repair.csv")
# print(df_repair.shape)
# df_repair.head(2)

In [ ]:
# cols_ = df_ksa.columns
#     df_['year'] = df_.periode.str[2:4].astype("int")
#     df_['nth'] = df_.nth.astype("int")

#     df_new = df_.merge(df_repair[['tahun', 'bulan', 'class','idsubsegmen_repair', 'idsubsegmen_old']], \
#               how='left', left_on=['year', 'nth', 'idsubsegment'],
#              right_on = ['tahun','bulan','idsubsegmen_old'])
    
#     df_new['idsubsegment'] = df_new.idsubsegmen_repair
#     df_new['observation'] = df_new['class']
#     df_new['kdpoint'] = df_new['idpoint'].str[-3:]
#     df_new['idpoint'] = df_new.idsubsegment + df_new['kdpoint']
#     df_new = df_new[cols_]

In [16]:
## list all MGRS
mgrs_all_ = list(df_ksa.MGRS.unique())

## Cek MGRS not done
mgrs_done_ = os.listdir('/data/ksa/03_Sampling/data/'+idprov+'/') 
mgrs_done_ = [x[-9:-4] for x in mgrs_done_]

mgrs_not_done_ = [x for x in mgrs_all_ if x not in mgrs_done_]
mgrs_not_done_

['49MEM', '49MFM', '49MGM', '49MHM', '49MFN']

In [17]:
for mgrs in mgrs_not_done_:
    df_result = pd.DataFrame()
    if(check_image(mgrs)):
        try:
            print(mgrs + " available")
            path_to_mgrs = "/data/ksa/01_Image_Acquisition/02_Processed_mosaic/"+mgrs+"/"
            all_sentinel_data = glob(path_to_mgrs+"*.dim")
            df_ksa_mgrs = df_ksa.loc[df_ksa.MGRS == mgrs]
            for sent in tqdm(all_sentinel_data):
                path_to_sentinel_data = sent
                periode = path_to_sentinel_data[-21:-4]
                product_subset,gc,bands_names = get_gc(path_to_sentinel_data)
                bands_names = [col for col in bands_names if 'VH' in col]
                df_ksa_mgrs_tmp = df_ksa_mgrs.copy()
                df_ksa_mgrs_tmp = df_ksa_mgrs_tmp.reset_index(drop=True)
                df_ksa_mgrs_tmp.loc[:, bands_names] = 0.0
                df_ksa_mgrs_tmp.loc[:, 'periode'] =periode  
                # print(bands_names)
                for i,r in df_ksa_mgrs_tmp.iterrows():
                    val = get_values(gc, bands_names, r['lat'], r['long'])
                    df_ksa_mgrs_tmp.loc[i,bands_names] = val
                    # break
                # break
                df_result = pd.concat([df_result,df_ksa_mgrs_tmp])
    
            df_result = df_result[['idpoint','MGRS','Sigma0_VH_db','periode']]
            with open('/data/ksa/03_Sampling/data/'+idprov+'/sampling_'+mgrs+'.pkl', 'wb') as f:
                pickle.dump(df_result, f)
            
            print("done sampling for : "+ mgrs)
            # break
        except: 
            print("Error on : "+ mgrs)
    else:
        print(mgrs + " not available")
        continue
    # break

49MEM available


  0%|          | 0/56 [00:00<?, ?it/s]Traceback (most recent call last):
  File "/tmp/ipykernel_28759/23781209.py", line 23, in get_values
    temp_band.readPixels(int(pixel_pos.x), int(pixel_pos.y), 1, 1, tmp)
RuntimeError: java.io.IOException: DimapProductReader: Unable to read file '/data/ksa/01_Image_Acquisition/02_Processed_mosaic/49MEM/20220606_20220617.data/Sigma0_VH_db.img' referenced by 'Sigma0_VH_db'.
Traceback (most recent call last):
  File "/tmp/ipykernel_28759/23781209.py", line 23, in get_values
    temp_band.readPixels(int(pixel_pos.x), int(pixel_pos.y), 1, 1, tmp)
RuntimeError: java.io.IOException: DimapProductReader: Unable to read file '/data/ksa/01_Image_Acquisition/02_Processed_mosaic/49MEM/20220606_20220617.data/Sigma0_VH_db_count.img' referenced by 'Sigma0_VH_db_count'.
  0%|          | 0/56 [00:01<?, ?it/s]

Sigma0_VH_db
5050 4287
1127 4287
java.io.IOException: DimapProductReader: Unable to read file '/data/ksa/01_Image_Acquisition/02_Processed_mosaic/49MEM/20220606_20220617.data/Sigma0_VH_db.img' referenced by 'Sigma0_VH_db'.
Sigma0_VH_db_count
5050 4287
1127 4287
java.io.IOException: DimapProductReader: Unable to read file '/data/ksa/01_Image_Acquisition/02_Processed_mosaic/49MEM/20220606_20220617.data/Sigma0_VH_db_count.img' referenced by 'Sigma0_VH_db_count'.
Error on : 49MEM
49MFM available



  0%|          | 0/56 [00:00<?, ?it/s]Traceback (most recent call last):
  File "/tmp/ipykernel_28759/23781209.py", line 23, in get_values
    temp_band.readPixels(int(pixel_pos.x), int(pixel_pos.y), 1, 1, tmp)
RuntimeError: java.io.IOException: DimapProductReader: Unable to read file '/data/ksa/01_Image_Acquisition/02_Processed_mosaic/49MFM/20220606_20220617.data/Sigma0_VH_db.img' referenced by 'Sigma0_VH_db'.
Traceback (most recent call last):
  File "/tmp/ipykernel_28759/23781209.py", line 23, in get_values
    temp_band.readPixels(int(pixel_pos.x), int(pixel_pos.y), 1, 1, tmp)
RuntimeError: java.io.IOException: DimapProductReader: Unable to read file '/data/ksa/01_Image_Acquisition/02_Processed_mosaic/49MFM/20220606_20220617.data/Sigma0_VH_db_count.img' referenced by 'Sigma0_VH_db_count'.
  0%|          | 0/56 [00:02<?, ?it/s]


Sigma0_VH_db
5058 4302
136 4302
java.io.IOException: DimapProductReader: Unable to read file '/data/ksa/01_Image_Acquisition/02_Processed_mosaic/49MFM/20220606_20220617.data/Sigma0_VH_db.img' referenced by 'Sigma0_VH_db'.
Sigma0_VH_db_count
5058 4302
136 4302
java.io.IOException: DimapProductReader: Unable to read file '/data/ksa/01_Image_Acquisition/02_Processed_mosaic/49MFM/20220606_20220617.data/Sigma0_VH_db_count.img' referenced by 'Sigma0_VH_db_count'.
Error on : 49MFM
49MGM available


...

Sigma0_VH_db
5064 4327
2217 4327
java.io.IOException: DimapProductReader: Unable to read file '/data/ksa/01_Image_Acquisition/02_Processed_mosaic/49MGM/20220606_20220617.data/Sigma0_VH_db.img' referenced by 'Sigma0_VH_db'.
Sigma0_VH_db_count
5064 4327
2217 4327
java.io.IOException: DimapProductReader: Unable to read file '/data/ksa/01_Image_Acquisition/02_Processed_mosaic/49MGM/20220606_20220617.data/Sigma0_VH_db_count.img' referenced by 'Sigma0_VH_db_count'.
Error on : 49MGM
49MHM available



  0%|          | 0/60 [00:00<?, ?it/s]Traceback (most recent call last):
  File "/tmp/ipykernel_28759/23781209.py", line 23, in get_values
    temp_band.readPixels(int(pixel_pos.x), int(pixel_pos.y), 1, 1, tmp)
RuntimeError: java.io.IOException: DimapProductReader: Unable to read file '/data/ksa/01_Image_Acquisition/02_Processed_mosaic/49MHM/20220606_20220617.data/Sigma0_VH_db.img' referenced by 'Sigma0_VH_db'.
Traceback (most recent call last):
  File "/tmp/ipykernel_28759/23781209.py", line 23, in get_values
    temp_band.readPixels(int(pixel_pos.x), int(pixel_pos.y), 1, 1, tmp)
RuntimeError: java.io.IOException: DimapProductReader: Unable to read file '/data/ksa/01_Image_Acquisition/02_Processed_mosaic/49MHM/20220606_20220617.data/Sigma0_VH_db_count.img' referenced by 'Sigma0_VH_db_count'.
  0%|          | 0/60 [00:04<?, ?it/s]

Sigma0_VH_db
1578 4337
672 4337
java.io.IOException: DimapProductReader: Unable to read file '/data/ksa/01_Image_Acquisition/02_Processed_mosaic/49MHM/20220606_20220617.data/Sigma0_VH_db.img' referenced by 'Sigma0_VH_db'.
Sigma0_VH_db_count
1578 4337
672 4337
java.io.IOException: DimapProductReader: Unable to read file '/data/ksa/01_Image_Acquisition/02_Processed_mosaic/49MHM/20220606_20220617.data/Sigma0_VH_db_count.img' referenced by 'Sigma0_VH_db_count'.
Error on : 49MHM
49MFN available



  0%|          | 0/56 [00:00<?, ?it/s]Traceback (most recent call last):
  File "/tmp/ipykernel_28759/23781209.py", line 23, in get_values
    temp_band.readPixels(int(pixel_pos.x), int(pixel_pos.y), 1, 1, tmp)
RuntimeError: java.io.IOException: DimapProductReader: Unable to read file '/data/ksa/01_Image_Acquisition/02_Processed_mosaic/49MFN/20220606_20220617.data/Sigma0_VH_db.img' referenced by 'Sigma0_VH_db'.
Traceback (most recent call last):
  File "/tmp/ipykernel_28759/23781209.py", line 23, in get_values
    temp_band.readPixels(int(pixel_pos.x), int(pixel_pos.y), 1, 1, tmp)
RuntimeError: java.io.IOException: DimapProductReader: Unable to read file '/data/ksa/01_Image_Acquisition/02_Processed_mosaic/49MFN/20220606_20220617.data/Sigma0_VH_db_count.img' referenced by 'Sigma0_VH_db_count'.
  0%|          | 0/56 [00:32<?, ?it/s]

Sigma0_VH_db
5049 5083
152 5083
java.io.IOException: DimapProductReader: Unable to read file '/data/ksa/01_Image_Acquisition/02_Processed_mosaic/49MFN/20220606_20220617.data/Sigma0_VH_db.img' referenced by 'Sigma0_VH_db'.
Sigma0_VH_db_count
5049 5083
152 5083
java.io.IOException: DimapProductReader: Unable to read file '/data/ksa/01_Image_Acquisition/02_Processed_mosaic/49MFN/20220606_20220617.data/Sigma0_VH_db_count.img' referenced by 'Sigma0_VH_db_count'.
Error on : 49MFN


In [18]:
## Cek error mgrs
mgrs_done_ = os.listdir('/data/ksa/03_Sampling/data/'+idprov+'/') 
mgrs_done_ = [x[-9:-4] for x in mgrs_done_]

mgrs_not_done_ = [x for x in mgrs_all_ if x not in mgrs_done_]
mgrs_not_done_

['49MEM', '49MFM', '49MGM', '49MHM', '49MFN']

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming df_ is your DataFrame
filtered_df = df_.query('idpoint == "321201001A1#02"').sort_values('periode')

plt.figure(figsize=(10, 6))
plt.plot(filtered_df['periode'], filtered_df['Sigma0_VH_db'], marker='o')

plt.title('Sigma0_VH_db over Time for idpoint "321201003B2#13"')
plt.xlabel('Periode')
plt.ylabel('Sigma0_VH_db')
plt.grid(True)
plt.xticks(rotation=45)  # Rotate x-axis labels if needed

plt.tight_layout()
plt.show()



In [ ]:
mgrs = '49LDM'
with open('/data/ksa/03_Sampling/data/'+idprov+'/sampling_'+mgrs+'.pkl', 'rb') as f:
    df_ = pickle.load(f)

df_.head()
# df_.to_csv("cek_"+mgrs+'.csv')

In [ ]:
df_.describe()